<a href="https://colab.research.google.com/github/mario-rot/Introduction-to-Human-Language-Technology/blob/main/IHLTscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%shell
git clone https://github.com/mario-rot/Introduction-to-Human-Language-Technology.git
cd 'Introduction-to-Human-Language-Technology'
mv 'Complementary Material' /content/
mv 'text_processing.py' /content/

pip install svgling

Cloning into 'Introduction-to-Human-Language-Technology'...
remote: Enumerating objects: 977, done.
remote: Counting objects: 100% (977/977), done.
remote: Compressing objects: 100% (882/882), done.
remote: Total 977 (delta 166), reused 865 (delta 90), pack-reused 0
Receiving objects: 100% (977/977), 2.72 MiB | 8.08 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
import nltk
import string
import math
from nltk.util import bigrams, trigrams
from nltk.corpus import wordnet as wn
### added so text_processing will work ###
nltk.download('wordnet_ic')
##########################################
from text_processing import text_processing, compute_metrics
from collections import Counter
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [6]:
dt = pd.read_csv('/content/Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [7]:
dt.head()

,0,1
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi..."


In [8]:
sentences1 = dt[0]
sentences2 = dt[1]

tp1 = text_processing(sentences1).lemmatize_data()
tp2 = text_processing(sentences2).lemmatize_data()


 -- Data hasn't been cleaned, to lemmatize the data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to lemmatize the data is going to be cleaned with the default parameters --



In [9]:
cln1 = text_processing(sentences1).clean_data()
cln2 = text_processing(sentences2).clean_data()

mcl1 = text_processing(sentences1).mc_lemmatize_data(lemma=True)
mcl2 = text_processing(sentences2).mc_lemmatize_data(lemma=True)

syn1 = [[lemma.synset() for lemma in sentence if lemma] for sentence in mcl1]
syn2 = [[lemma.synset() for lemma in sentence if lemma] for sentence in mcl2]


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --



In [10]:
mcl1[95]

[Lemma('addition.n.03.increase'),
 Lemma('power.n.01.power'),
 Lemma('large.a.01.big'),
 Lemma('state.n.04.country'),
 Lemma('expense.v.01.expense'),
 Lemma('small.a.01.small'),
 Lemma('state.n.04.country')]

In [11]:
def clean(corpus, stopwords, minwords_len, signs):
   corpus = corpus.split(' ')
   corpus = [word.lower() for word in corpus]
   corpus = [word if not any(caracter in signs for caracter in word) else remove_signs(word) for word in corpus]
   corpus = [word for word in corpus if word not in stopwords and word.isalpha()]
   corpus = [word for word in corpus if len(word) > minwords_len]
   return corpus

def remove_signs(wrd):
  wrd = list(wrd)
  wrd = [word for word in wrd if not any(caracter in signs for caracter in word)]
  wrd = ''.join(wrd)
  return wrd


stopwords=set(nltk.corpus.stopwords.words('english'))
signs = string.punctuation
minwords_len = 2

In [12]:
def cosine_similarity(sentence1, sentence2):
  sim = 0
  all_words = set(sentence1).union(set(sentence2))
  s1vec = []
  s2vec = []
  for word in all_words:
    if word in sentence1:
      s1vec.append(1)
    else:
      s1vec.append(0)
    if word in sentence2:
      s2vec.append(1)
    else:
      s2vec.append(0)
  sim = np.dot(np.array(s1vec), np.array(s2vec)) / (math.sqrt(sum(s1vec)*sum(s2vec)))
  print(sentence1)
  print(sentence2)
  print(all_words)
  print(s1vec)
  print(s2vec)
  print(sim)
  return sim

In [13]:
sent1 = clean(dt[0][0], stopwords, minwords_len, signs)
sent2 = clean(dt[1][0], stopwords, minwords_len, signs)

In [14]:
cosine_similarity(sent1, sent2)

['leaders', 'given', 'new', 'chance', 'let', 'hope', 'seize']
['leaders', 'benefit', 'aujourd', 'hui', 'new', 'luck', 'lets', 'let', 'therefore', 'seize']
{'let', 'therefore', 'seize', 'aujourd', 'hui', 'lets', 'luck', 'benefit', 'hope', 'given', 'chance', 'leaders', 'new'}
[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
0.47809144373375745


0.47809144373375745

In [15]:
def unigram_similarity(words1, words2):
    count_same = 0
    for w in words1:
        count_same += min(words1.count(w), words2.count(w))
    
    if len(words1) > 0 or len(words2) > 0:
        return 2*count_same/(len(words1)+len(words2))
    else:
        return 0

In [16]:
unigram_similarity(sent1, sent2)

0.47058823529411764

In [17]:
8/17

0.47058823529411764

In [18]:
def unigram_similarity_mine(sentence1, sentence2, remove_duplicates):
  if remove_duplicates:
    sentence1 = list(set(sentence1))
    sentence2 = list(set(sentence2))
    print(sentence1)
    print(sentence2)
  total_words = len(sentence1) + len(sentence2)
  used_words = []
  count = 0
  for word in sentence1:
    if word in sentence2 and word not in used_words:
      count += sentence1.count(word)
      count += sentence2.count(word)
    used_words.append(word)

  sim = count / total_words
  return sim

In [ ]:
def unigram_similarity_mine(sentence1, sentence2, remove_duplicates, consider_importance):
  if remove_duplicates:
    sentence1 = list(set(sentence1))
    sentence2 = list(set(sentence2))
    print(sentence1)
    print(sentence2)

  if consider_importance:
    idf = get_idf(sentences)
  else:
    idf = 1


  total_words = len(sentence1) + len(sentence2)
  used_words = []
  count = 0
  for word in sentence1:
    if word in sentence2 and word not in used_words:
      if consider_importance:
        tf1 = float(sentence1.count(word)) / len(sentence1)
        tf2 = float(sentence2.count(word)) / len(sentence2)
        count += (sentence1.count(word) * (tf1 * idf))
        count += (sentence2.count(word) * (tf2 * idf))
      else:
        count += sentence1.count(word)
        count += sentence2.count(word)
    used_words.append(word)

  sim = count / total_words
  return sim

In [19]:
unigram_similarity_mine(sent1, sent2, False)

0.47058823529411764

In [20]:
dt[0][95]

'It increases the power of the big countries at the expense of the small countries.'

In [21]:
sentence1 = 'It increases the power of the big countries at the expense of the small countries.'
sentence2 = 'This has the effect of increasing the power of large countries at the expense of smaller countries.'


sent1 = clean(sentence1, stopwords, minwords_len, signs)
sent2 = clean(sentence2, stopwords, minwords_len, signs)

print(unigram_similarity(sent1, sent2))
print(unigram_similarity_mine(sent1, sent2, False))
print(sent1)
print(sent2)
print(unigram_similarity_mine(sent1, sent2, True))

0.8
0.5333333333333333
['increases', 'power', 'big', 'countries', 'expense', 'small', 'countries']
['effect', 'increasing', 'power', 'large', 'countries', 'expense', 'smaller', 'countries']
['small', 'increases', 'big', 'expense', 'power', 'countries']
['increasing', 'large', 'smaller', 'expense', 'effect', 'power', 'countries']
0.46153846153846156


In [22]:
print(unigram_similarity_mine(tp1[95], tp2[95], True))
print(unigram_similarity_mine(tp1[95], tp2[95], False))

['small', 'big', 'expense', 'country', 'increase', 'power']
['small', 'large', 'expense', 'country', 'increase', 'effect', 'power']
0.7692307692307693
0.7857142857142857


In [23]:
print(mcl1[95])
print(mcl2[95])

[Lemma('addition.n.03.increase'), Lemma('power.n.01.power'), Lemma('large.a.01.big'), Lemma('state.n.04.country'), Lemma('expense.v.01.expense'), Lemma('small.a.01.small'), Lemma('state.n.04.country')]
[Lemma('consequence.n.01.effect'), Lemma('increase.v.01.increase'), Lemma('power.n.01.power'), Lemma('large.a.01.large'), Lemma('state.n.04.country'), Lemma('expense.v.01.expense'), Lemma('small.a.01.small')]


In [24]:
mcl195 = [lemma for lemma in mcl1[0] if lemma]
mcl295 = [lemma for lemma in mcl2[0] if lemma]



print(mcl195)
print(mcl295)

print(sentences2[0])

print(unigram_similarity_mine(mcl195, mcl295, True))
print(unigram_similarity_mine(mcl195, mcl295, False))

[Lemma('leader.n.01.leader'), Lemma('give.v.01.give'), Lemma('new.a.01.new'), Lemma('opportunity.n.01.chance'), Lemma('let.v.01.let'), Lemma('seize.v.01.seize')]
[Lemma('leader.n.01.leader'), Lemma('profit.v.01.benefit'), Lemma('new.a.01.new'), Lemma('fortune.n.04.fortune'), Lemma('lashkar-e-taiba.n.01.Lashkar-e-Taiba'), Lemma('let.v.01.let'), Lemma('therefore.r.01.therefore'), Lemma('seize.v.01.seize')]
The leaders benefit aujourd' hui of a new luck and let's let them therefore seize it.
[Lemma('let.v.01.let'), Lemma('seize.v.01.seize'), Lemma('opportunity.n.01.chance'), Lemma('leader.n.01.leader'), Lemma('new.a.01.new'), Lemma('give.v.01.give')]
[Lemma('profit.v.01.benefit'), Lemma('let.v.01.let'), Lemma('therefore.r.01.therefore'), Lemma('seize.v.01.seize'), Lemma('new.a.01.new'), Lemma('leader.n.01.leader'), Lemma('lashkar-e-taiba.n.01.Lashkar-e-Taiba'), Lemma('fortune.n.04.fortune')]
0.5714285714285714
0.5714285714285714


In [25]:

# conclusion: higher similarity scores with duplicates not removed

counter = 0
counter_eq = 0
for pair in zip(tp1, tp2):
  no_dups = unigram_similarity_mine(pair[0], pair[1], True)
  w_dups = unigram_similarity_mine(pair[0], pair[1], False)
  if no_dups > w_dups:
    counter +=1
  elif no_dups == w_dups:
    counter_eq +=1

print(counter)
print(counter_eq)
print(len(tp1) - counter - counter_eq)



['let', 'seize', 'hope', 'chance', 'leader', 'new', 'give']
['benefit', 'let', 'aujourd', 'therefore', 'seize', 'hui', 'leader', 'luck', 'new']
['amendment', 'change', 'paragraph', 'certain', 'propose', 'reference']
['amendment', 'change', 'paragraph', 'certain', 'propose', 'reference']
['let', 'ally', 'fervent', 'tax', 'remind', 'supporter', 'include']
['among', 'ally', 'strong', 'like', 'tax', 'remind', 'would']
['take', 'place', 'vote', 'today']
['take', 'place', 'vote']
['disappointed', 'inactive', 'fisherman', 'tire']
['disappointed', 'inactive', 'fisherman', 'tire']
['within', 'qualified', 'revert', 'house', 'article', 'majority', 'neither']
['parliament', 'voting', 'article', 'majority', 'back']
['small', 'big', 'expense', 'country', 'increase', 'power']
['detriment', 'augment', 'baby', 'big', 'potency', 'country', 'effect']
['disappointed', 'inactive', 'fisherman', 'tire']
['inactive', 'disappointed', 'fisher', 'tire']
['let', 'seize', 'hope', 'chance', 'leader', 'new', 'give']

In [26]:
# the best option is to use synsets, as the most frequent occurrence is that all methods are equal
# but the second best option is synsets -- therefore, using synsets we will have the highest value
# (189 + 175) / 459 of the time for unigrams
# (266 + 116) / 459 of the time for bigrams
# (379 + 48) / 459 of the time for trigrams -- likely because there are fewer trigrams so 0 == 0

results = []
clean_counter = 0
lemma_counter = 0
syns_counter = 0
all_equal_ctr = 0
cln_lem_ctr = 0
lem_syns_ctr = 0
cln_syns_ctr = 0
for n,(c1, c2, m1, m2, s1, s2) in enumerate(zip(cln1, cln2, tp1, tp2, syn1, syn2)):
  print(n)
  clean = trigram_similarity(c1, c2, False)
  lemma = trigram_similarity(m1, m2, False)
  syns = trigram_similarity(s1, s2, False)
  if clean > lemma and clean > syns:
    clean_counter +=1
  elif lemma > clean and lemma > syns:
    lemma_counter += 1
  elif syns > lemma and syns > clean:
    syns_counter +=1
  elif clean == lemma == syns:
    all_equal_ctr += 1
  elif clean == lemma:
    cln_lem_ctr +=1
  elif lemma == syns:
    lem_syns_ctr += 1
  elif clean == syns:
    cln_syns_ctr += 1

  results.append(np.array([clean, lemma, syns]).argmax())

print(Counter(results))
print(clean_counter, lemma_counter, syns_counter)
print(all_equal_ctr, cln_lem_ctr, lem_syns_ctr, cln_syns_ctr)

sum([all_equal_ctr, cln_lem_ctr, lem_syns_ctr, cln_syns_ctr, clean_counter, lemma_counter, syns_counter])

0


NameError: ignored

In [ ]:
clean = unigram_similarity_mine(cln1[90], cln2[90], False)
lemma = unigram_similarity_mine(tp1[90], tp2[90], False)
syns = unigram_similarity_mine(syn1[90], syn2[90], False)

In [ ]:
clean,lemma,syns

In [ ]:
10/13

In [ ]:
9/16

In [27]:
6/13

0.46153846153846156

In [28]:
def bigram_similarity(sentence1, sentence2, remove_duplicates):
  bigrams1 = list(nltk.bigrams(sentence1))
  bigrams2 = list(nltk.bigrams(sentence2))
  if remove_duplicates:
    bigrams1 = list(set(bigrams1))
    bigrams2 = list(set(bigrams2))
  total_bigrams = len(bigrams1) + len(bigrams2)
  used_bigrams = []
  count = 0
  for bigram in bigrams1:
    if bigram in bigrams2 and bigram not in used_bigrams:
      count += bigrams1.count(bigram)
      count += bigrams2.count(bigram)
    used_bigrams.append(bigram)
  print(bigrams1)
  print(bigrams2)
  print(total_bigrams)
  if total_bigrams == 0:
    return 0
  else:
    sim = count / total_bigrams
    return sim

In [29]:
bigram_similarity(sent1, sent2, False)

[('increases', 'power'), ('power', 'big'), ('big', 'countries'), ('countries', 'expense'), ('expense', 'small'), ('small', 'countries')]
[('effect', 'increasing'), ('increasing', 'power'), ('power', 'large'), ('large', 'countries'), ('countries', 'expense'), ('expense', 'smaller'), ('smaller', 'countries')]
13


0.15384615384615385

In [30]:
2/15

0.13333333333333333

In [31]:
def trigram_similarity(sentence1, sentence2, remove_duplicates):
  trigrams1 = list(nltk.trigrams(sentence1))
  trigrams2 = list(nltk.trigrams(sentence2))
  if remove_duplicates:
    trigrams1 = list(set(trigrams1))
    trigrams2 = list(set(trigrams2))
  total_trigrams = len(trigrams1) + len(trigrams2)
  used_trigrams = []
  count = 0
  for trigram in trigrams1:
    if trigram in trigrams2 and trigram not in used_trigrams:
      count += trigrams1.count(trigram)
      count += trigrams2.count(trigram)
    used_trigrams.append(trigram)
  print(trigrams1)
  print(trigrams2)
  print(count)
  if total_trigrams == 0:
    return 0
  else:
    sim = count / total_trigrams
    return sim

In [32]:
trigram_similarity(sent1, sent2, False)

[('increases', 'power', 'big'), ('power', 'big', 'countries'), ('big', 'countries', 'expense'), ('countries', 'expense', 'small'), ('expense', 'small', 'countries')]
[('effect', 'increasing', 'power'), ('increasing', 'power', 'large'), ('power', 'large', 'countries'), ('large', 'countries', 'expense'), ('countries', 'expense', 'smaller'), ('expense', 'smaller', 'countries')]
0


0.0

In [33]:
2/15

0.13333333333333333

In [53]:
def unigram_similarity_importance(sentence1, sentence2, remove_duplicates):
  idf = get_idf(dt[0] + dt[1])  # need this to be the final cleaned sentences

  if remove_duplicates:
    sentence1 = list(set(sentence1))
    sentence2 = list(set(sentence2))

  total_words = len(sentence1) + len(sentence2)
  used_words = []
  count = 0
  for word in sentence1:
    if word in sentence2 and word not in used_words:
      count += sentence1.count(word)
      count += sentence2.count(word)
    used_words.append(word)

  sim = count / total_words

  return sim

new code as of 12/15

In [78]:
def get_idf(sentences):
  counterDict = {}
  total_words = 0

  for sentence in sentences:
    for word in sentence:
      if word in counterDict.keys():
        counterDict[word] += 1
      else:
        counterDict[word] = 1
      total_words += 1
  
  idf = {}
  for word in counterDict.keys():
    idf[word] = math.log(float(len(sentences)) / float(counterDict[word])) 
  return idf

In [79]:
idf = get_idf(tp1 + tp2)
print(min(idf.values()))
print(max(idf.values()))

2.257849199152654
6.822197390620491


In [112]:
def unigram_similarity_mine(sentence1, sentence2, remove_duplicates, consider_importance, corpus):
  if remove_duplicates:
    sentence1 = list(set(sentence1))
    sentence2 = list(set(sentence2))

  if consider_importance:
    idf = get_idf(corpus)

  total_words = len(sentence1) + len(sentence2)
  used_words = []
  count = 0
  importance_vals = []
  for word in sentence1:
    if word in sentence2 and word not in used_words:
      if consider_importance:
        tf1 = float(sentence1.count(word)) / len(sentence1)
        tf2 = float(sentence2.count(word)) / len(sentence2)
        importance1 = tf1 * idf[word]
        importance2 = tf2 * idf[word]
        importance_vals.append(importance1)
        importance_vals.append(importance2)
        count += (sentence1.count(word) * importance1)
        count += (sentence2.count(word) * importance2)
      else:
        count += sentence1.count(word)
        count += sentence2.count(word)
        importance_vals.append(1)
    used_words.append(word)

  if importance_vals == []:
    importance_vals = [1]
  max_importance = max(importance_vals)
  #print(max_importance)
  sim = (count / max_importance) / total_words
  print(sim)
  return sim

In [113]:
corpus = tp1+tp2
for i in range(459):
  unigram_similarity_mine(tp1[i], tp2[i], True, True, corpus)

0.391982344488564
0.9392995144631895
0.41541705794439915
0.7044918319588901
0.9931240849675144
0.28443193250623927
0.24792172525846246
0.7454160566450097
0.3897365131235611
0.7044918319588901
0.6607690932262013
0.5688340177975626
0.7149084935968805
0.41319836076480376
0.16666666666666663
1.0
0.41665145173146856
0.7466338228288781
0.5774455966173675
0.7743086544535073
0.31804733849790523
1.0
0.5688340177975626
1.0
0.9387523422525998
0.3856952921394077
0.2857142857142857
0.25
0.6656422767775366
0.6089511189318298
0.3333333333333333
0.5
0.33333333333333337
0.31804733849790523
0.5219581019415684
0.7108753456017526
0.3333333333333333
0.45842030420274865
0.9392995144631895
0.8394557693504157
0.3021407850700522
1.0
0.7804804538376467
0.4648481558604042
0.3261242526605621
0.7044918319588901
0.6192024684901926
0.3527841100397076
1.0
0.7149084935968805
1.0
0.5774455966173675
0.41665145173146856
0.28443193250623927
0.5255781038670186
0.7466338228288781
0.8848750969147173
0.2
0.7129630572160005
0.

In [117]:
tp1[452]

['perhaps', 'could', 'avoid', 'catastrophe']

In [118]:
tp2[452]

['might', 'able', 'prevent', 'disaster']

In [122]:
unigram_similarity_mine(tp1[451],tp2[451], True, False, corpus)

0.6


0.6